# Institute for Rock Magnetics Kappa MagIC 

The purpose of this notebook is to provide users an analytical package wholly contained in a web-based Jupyter environment.  It is designed to allow dynamic exploration of experimental data from the MagIC database.

It is intended that this be run cell by cell allowing users to select corrections/filters as needed.

Key components here include portions of notebooks, and functions developed for the EarthRef JupyterHub:
https://www2.earthref.org/MagIC/jupyter-notebooks

This was built with the following dataset:

R. Doctor, J. M. Feinberg (2022). Differential thermal analysis using high temperature susceptibility instruments. Journal of Geophysical Research: Solid Earth. doi:10.1029/2021JB023789.

Paper (.pdf open-access)
https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1029/2021JB023789

A way that we can readily search for a contribution on MagIC is using the study doi. Let's go check out the contribution on the MagIC website:

http://dx.doi.org/10.7288/V4/MAGIC/19369

## Analyzing and visualizing the data

We want to be able to get the data from MagIC, import the data into our notebook, and have a look at it.

To start with, let's import some functions from PmagPy:

In [ ]:
import pmagpy.ipmag as ipmag

## Import the data

Run the cell below, it is configured with the example datset, but if you would like to change the magic-id you can do so prior to clicking the button.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the default value for magic_id
magic_id = '19369'
magic_file_name = None  # Initialize magic_file_name as None

# Create a text box widget with the default value
magic_id_textbox = widgets.Text(value=magic_id, description='Magic ID:')

# Function to handle changes in the text box value
def update_magic_id(change):
    global magic_id
    magic_id = change.new

# Attach the event handler to the text box
magic_id_textbox.observe(update_magic_id, names='value')

# Display the text box
display(magic_id_textbox)

# Now you can access the current value of magic_id, and it will update when users change it in the text box

# Add a button for users to trigger the download
download_button = widgets.Button(description="Download Magic File")

# Function to handle the button click event and trigger the download
def download_magic_file(button_click):
    global magic_file_name  # Declare magic_file_name as global
    result, magic_file_name = ipmag.download_magic_from_id(magic_id)
    print(f"Downloaded Magic File: {magic_file_name}")

# Attach the event handler to the download button
download_button.on_click(download_magic_file)

# Display the download button
display(download_button)


Running this function should give you a file called ```magic_contribution_19369.txt``` in the folder that this notebook is in. Let's go find that file in the Jupyter directory and open it to have a look.

In the above code cell, we saved a variable `magic_file_name` that is the name of the files that was downloaded.

In [ ]:
magic_file_name

## Unpacking the tables

A MagIC contribution is a single .txt file that comprises a number of tables. In the case of this contribution, we have these tables:
- contribution
- locations
- sites
- samples
- specimens
- measurements
- criteria
- ages

We want unpack the contribution into these distinct tables.

In [ ]:
ipmag.download_magic(magic_file_name,print_progress=False)

## Importing specific MagIC tables

For some other functions in `PmagPy` data need to be imported to be Python objects. There is a really nice package for dealing with tabular data in Python called `pandas`. The code cell below imports this package so that we can use it. We use the typical scientific Python nomenclature of importing it for use to the shorthand `pd`.

In [ ]:
import pandas as pd

## Import the sites table

We can now use pandas to import the sites table to a pandas dataframe using the function `pd.read_csv()`.

In [ ]:
sites = pd.read_csv('sites.txt',sep='\t',header=1)
sites

We can extract specific columns from the dataframe by using the nomenclature `dataframe_name['column_name']`. In this case, the dataframe name is `sites` and the column name might be `suc_chi_mass`.

## Specimen Table
Print a list of specimens from this contribution.

In [ ]:
specimens = pd.read_csv('specimens.txt',sep='\t',header=1)
specimens

## Select the measurements

Select the specimen using the "specimen_to_filter" variable:

Example datasets include: 
BHRS13-2B5L1A_01 ("Tidy")
BHRS13-2B5L1B_01 ("Messy")

You may change the selection by editing the code below:

<font color="teal">**# Define the variable for the condition**</font>

**specimen_to_filter =** <font color="red">'BHRS13-2B5L1A_01'</font>




In [ ]:
import plotly.express as px
import ipywidgets as widgets
import pandas as pd
from IPython.display import display, HTML  # Add HTML to the import statement

# Define global variables for filtered_df and specimen_to_filter
filtered_df = None
specimen_to_filter = 'BHRS13-2B5L1A_01'  # Default value

# Read the data from the "measurements.txt" file into a DataFrame
df = pd.read_csv("measurements.txt", sep="\t", header=1)

# Create a text input widget with the default value and wrapped description
text_input = widgets.Text(
    value=specimen_to_filter,
    description='Specimen to filter:',
    style={'description_width': 'initial'}  # Wrap the description
)

# Create a button widget
apply_button = widgets.Button(description='Apply Filter')

# Create an output widget for displaying the filtered DataFrame with a scrollable window
output = widgets.Output(layout={'max_height': '150px', 'overflow_y': 'scroll'})

# Define a function to update the filtered DataFrame
def filter_dataframe(b):
    global filtered_df, specimen_to_filter  # Declare that we want to use the global variables

    with output:
        output.clear_output()
        # Get the value from the text input
        specimen_to_filter = text_input.value

        # Filter the DataFrame based on the "specimen" column condition using the variable
        filtered_df = df[df['specimen'] == specimen_to_filter]

        # Display the filtered DataFrame
        display(HTML(filtered_df.to_html(classes='scrollable', index=False)))

# Set the callback function for the button click event
apply_button.on_click(filter_dataframe)

# Display the widgets
display(text_input, apply_button, output)


## Get the Max meas_temp

Find the maximum meas_temp, we'll use this to define the heating and cooling tick marks and create the filtered dataframe.

In [ ]:
from IPython.display import HTML, display

# Find the maximum value of 'meas_temp'
max_meas_temp = filtered_df['meas_temp'].max()

# Filter the DataFrame to get rows where 'meas_temp' is equal to the maximum value
rows_with_max_meas_temp = filtered_df.loc[filtered_df['meas_temp'] == max_meas_temp]

# Create an HTML table for the DataFrame
html_table = rows_with_max_meas_temp.to_html(classes='scrollable', escape=False)

# Display the HTML table with scrollable frame
display(HTML(html_table))

## Install Plotly
This tool is installed to allow for dynamic plotting.

In [ ]:
!pip install --user plotly

In [ ]:
from IPython.display import HTML, display

# Find the maximum value of 'meas_temp'
max_meas_temp = filtered_df['meas_temp'].max()

# Filter the DataFrame to get rows where 'meas_temp' is equal to the maximum value
rows_with_max_meas_temp = filtered_df.loc[filtered_df['meas_temp'] == max_meas_temp]

# Create an HTML table for the DataFrame
html_table = rows_with_max_meas_temp.to_html(classes='scrollable', escape=False)

# Display the HTML table with scrollable frame
display(HTML(html_table))


## Plot the data

This code generates a scatter plot that shows how the magnetic susceptibility ('susc_chi_mass') varies with temperature ('meas_temp') over the course of an experiment. The plot uses color to distinguish between different phases of the experiment, with 'red' representing the heating phase and 'blue' representing the cooling phase

In [ ]:
import plotly.express as px

# Assuming you have a DataFrame named 'filtered_df'

# Find the maximum value of 'meas_temp'
max_meas_temp = filtered_df['meas_temp'].max()

# Create a new column in the DataFrame to specify marker colors
filtered_df['marker_color'] = 'red'  # Default color is red

# Set color to blue for values with an index greater than the max index
filtered_df.loc[filtered_df.index > filtered_df[filtered_df['meas_temp'] == max_meas_temp].index[0], 'marker_color'] = 'blue'

# Set color back to red for the maximum 'meas_temp' value
filtered_df.loc[filtered_df['meas_temp'] == max_meas_temp, 'marker_color'] = 'red'

# Create a Plotly scatter plot with color mapping
fig = px.scatter(
    filtered_df,
    x='meas_temp',
    y='susc_chi_mass',
    color='marker_color',
    color_discrete_map={'red': 'red', 'blue': 'blue'},
    title=f"Magnetic Susceptibility vs Temperature ({specimen_to_filter})",
    labels={'meas_temp': 'Temperature (°K)', 'susc_chi_mass': 'Magnetic Susceptibility χ [m<sup>3</sup> kg<sup>-1</sup>]'}
)

# Customize the legend labels
fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
fig.update_traces(showlegend=True, selector=dict(marker_color='red'), name='Heating')
fig.update_traces(showlegend=True, selector=dict(marker_color='blue'), name='Cooling')

# Customize the y-axis to display decimal units
fig.update_layout(yaxis=dict(tickformat='.2e'))  # Set the desired decimal format

# Customize the legend title
fig.update_layout(legend_title_text='Experiment Phase')

# Show the Plotly figure
fig.show()


## Background correction:

Tools to perform background subtraction and smoothing:
1. Select lowest χ and subtract from selection
2. Manual input of value to subtract across the series
3. Blank datset
4. Select range, determine average, subtract that.
5. Running Average

** Not yet - 6. LOESS Smoothing

[Skip Background Correction and Smoothing Steps](#plotting)


### 1. Subtract minimum χ

This subtracts the minimum χ and creates a text file "min_χ_subtracted.txt" which can be used in functions below.

In [ ]:
import plotly.express as px
import pandas as pd

# Assuming you have a DataFrame named 'filtered_df'

# Create a copy of the original DataFrame
new_df = filtered_df.copy()

# Find the maximum value of 'meas_temp' in the copied DataFrame
max_meas_temp = new_df['meas_temp'].max()

# Find the minimum value of 'susc_chi_mass' in the copied DataFrame
min_susc_chi_mass = new_df['susc_chi_mass'].min()

# Subtract the minimum 'susc_chi_mass' from the 'susc_chi_mass' series in the copied DataFrame
new_df['susc_chi_mass'] = new_df['susc_chi_mass'] - min_susc_chi_mass

# Save the modified DataFrame to a new file
new_df.to_csv("min_χ_subtracted.txt", index=False)

# Create a new column in the copied DataFrame to specify marker colors
new_df['marker_color'] = 'red'  # Default color is red

# Find the index of the rows with the maximum 'meas_temp' value in the copied DataFrame
max_meas_temp_index = new_df.index[new_df['meas_temp'] == max_meas_temp].tolist()

# Set color to blue for values with an index greater than the max index in the copied DataFrame
new_df.loc[new_df.index > max_meas_temp_index[0], 'marker_color'] = 'blue'

# Create a Plotly scatter plot with color mapping using the copied DataFrame
fig = px.scatter(
    new_df,
    x='meas_temp',
    y='susc_chi_mass',
    color='marker_color',
    color_discrete_map={'red': 'red', 'blue': 'blue'},
    title=f"Magnetic Susceptibility -minimumχ vs Temperature ({specimen_to_filter})",
    labels={'meas_temp': 'Temperature (°K)', 'susc_chi_mass': 'Magnetic Susceptibility χ -minimumχ [m<sup>3</sup> kg<sup>-1</sup>]'}
)

# Customize the legend labels
fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
fig.update_traces(showlegend=True, selector=dict(marker_color='red'), name='Heating')
fig.update_traces(showlegend=True, selector=dict(marker_color='blue'), name='Cooling')

# Customize the y-axis to display decimal units
fig.update_layout(yaxis=dict(tickformat=".2e"))  # Set the desired decimal format

# Customize the legend title
fig.update_layout(legend_title_text='Experiment Phase')

# Show the Plotly figure
fig.show()


### 2. Subtract given value

This subtracts a given χ value as entered by the user.  The result is saved as "[value]_subtracted.txt".

In [ ]:
import plotly.express as px
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Assuming you have a DataFrame named 'filtered_df'

# Create a text box for entering the value to subtract
value_to_subtract_textbox = widgets.FloatText(
    value=10,  # Default value, you can change this
    description='Value to Subtract:',
    style={'description_width': 'initial'},
)

# Display the text box for user input
display(value_to_subtract_textbox)

# Define a function to update the DataFrame and save it
def update_and_save_data(value_to_subtract):
    # Make a copy of the original DataFrame to avoid modifying the original data
    modified_df = filtered_df.copy()
    
    # Subtract the specified value from the 'susc_chi_mass' series
    modified_df['susc_chi_mass'] = modified_df['susc_chi_mass'] - value_to_subtract

    # Find the maximum value of 'meas_temp'
    max_meas_temp = modified_df['meas_temp'].max()

    # Create a new column in the DataFrame to specify marker colors
    modified_df['marker_color'] = 'red'  # Default color is red

    # Find the index of the rows with the maximum 'meas_temp' value
    max_meas_temp_index = modified_df.index[modified_df['meas_temp'] == max_meas_temp].tolist()

    # Set color to blue for values with an index greater than the max index
    modified_df.loc[modified_df.index > max_meas_temp_index[0], 'marker_color'] = 'blue'

    # Create a Plotly scatter plot with color mapping
    fig = px.scatter(
        modified_df,
        x='meas_temp',
        y='susc_chi_mass',
        color='marker_color',
        color_discrete_map={'red': 'red', 'blue': 'blue'},
        title=f"Magnetic Susceptibility vs Temperature ({specimen_to_filter})",
        labels={'meas_temp': 'Temperature (°K)', 'susc_chi_mass': 'Magnetic Susceptibility χ [m<sup>3</sup> kg<sup>-1</sup>]'}
    )

    # Customize the legend labels
    fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
    fig.update_traces(showlegend=True, selector=dict(marker_color='red'), name='Heating')
    fig.update_traces(showlegend=True, selector=dict(marker_color='blue'), name='Cooling')

    # Customize the y-axis to display decimal units
    fig.update_layout(yaxis=dict(tickformat=".2e"))  # Set the desired decimal format

    # Customize the legend title
    fig.update_layout(legend_title_text='Experiment Phase')

    # Show the Plotly figure
    fig.show()

    # Save the modified DataFrame to a file with the specified value as the filename prefix
    modified_df.to_csv(f"{value_to_subtract}_subtracted.txt", index=False)

# Add a button to trigger the update and save function
update_button = widgets.Button(description="Update and Save")
display(update_button)

# Define a callback for the button click event
def on_button_click(b):
    value_to_subtract = value_to_subtract_textbox.value
    update_and_save_data(value_to_subtract)

# Attach the callback to the button click event
update_button.on_click(on_button_click)


### 3. Determine average for temperature range (to be used for subtraction ^)

Set range, determine average χ

In [ ]:
import plotly.express as px

# Assuming you have a DataFrame named 'filtered_df'

# Find the maximum value of 'meas_temp'
max_meas_temp = filtered_df['meas_temp'].max()

# Create a new column in the DataFrame to specify marker colors
filtered_df['marker_color'] = 'red'  # Default color is red

# Set color to blue for values with an index greater than the max index
filtered_df.loc[filtered_df.index > filtered_df[filtered_df['meas_temp'] == max_meas_temp].index[0], 'marker_color'] = 'blue'

# Set color back to red for the maximum 'meas_temp' value
filtered_df.loc[filtered_df['meas_temp'] == max_meas_temp, 'marker_color'] = 'red'

# Create a Plotly scatter plot with color mapping
fig = px.scatter(
    filtered_df,
    x='meas_temp',
    y='susc_chi_mass',
    color='marker_color',
    color_discrete_map={'red': 'red', 'blue': 'blue'},
    title=f"Magnetic Susceptibility vs Temperature",
    labels={'meas_temp': 'Temperature (°K)', 'susc_chi_mass': 'Magnetic Susceptibility χ [m<sup>3</sup> kg<sup>-1</sup>]'}
)

# Customize the legend labels
fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
fig.update_traces(showlegend=True, selector=dict(marker_color='red'), name='Heating')
fig.update_traces(showlegend=True, selector=dict(marker_color='blue'), name='Cooling')

# Customize the y-axis to display decimal units
fig.update_layout(yaxis=dict(tickformat=".2e"))  # Set the desired decimal format

# Customize the legend title
fig.update_layout(legend_title_text='Experiment Phase')

# Define a function to calculate average susc_chi_mass over a temperature range
def calculate_average_susc_chi(temp_range):
    filtered_data = filtered_df[(filtered_df['meas_temp'] >= temp_range[0]) & (filtered_df['meas_temp'] <= temp_range[1])]
    average_susc_chi = filtered_data['susc_chi_mass'].mean()
    return average_susc_chi

# Input boxes for temperature range
temp_range_start = float(input("Enter the starting temperature (°K): "))
temp_range_end = float(input("Enter the ending temperature (°K): "))

# Calculate the average susc_chi_mass over the specified range
average_susc_chi = calculate_average_susc_chi([temp_range_start, temp_range_end])
print(f'Average susc_chi_mass over the temperature range [{temp_range_start}, {temp_range_end}]: {average_susc_chi:.6f}')

# Show the Plotly figure
fig.show()


### Smooth the temperature data with a running average:

This code generates a scatter plot of the data smoothed with a running average.  Run this if you'd like to smooth your data. [colors need to be updated -DG]

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

# Define a running average smoothing function for a specific column
def running_average_smoothing(df, column_name, window_size):
    df['smoothed_' + column_name] = df[column_name].rolling(window=window_size, center=True, min_periods=1).mean()
    return df

# Load your data into a DataFrame, assuming you have a DataFrame named 'filtered_df'

# Get the window size from user input
window_size = int(input("Enter the window size for smoothing: "))

# Apply the running average smoothing to 'susc_chi_mass' separately
filtered_df = running_average_smoothing(filtered_df, 'susc_chi_mass', window_size)

# Find the maximum value of 'meas_temp'
max_meas_temp = filtered_df['meas_temp'].max()

# Create a new column in the DataFrame to specify marker colors
filtered_df['marker_color'] = 'red'  # Default color is red

# Set color to blue for values with an index greater than the max index
filtered_df.loc[filtered_df.index > filtered_df[filtered_df['meas_temp'] == max_meas_temp].index[0], 'marker_color'] = 'blue'

# Set color back to red for the maximum 'meas_temp' value
filtered_df.loc[filtered_df['meas_temp'] == max_meas_temp, 'marker_color'] = 'red'

# Create a Plotly figure with lines for smoothed data
fig = go.Figure()

# Create scatter plots for original data with color mapping
fig.add_trace(go.Scatter(
    x=filtered_df[filtered_df['marker_color'] == 'red']['meas_temp'],
    y=filtered_df[filtered_df['marker_color'] == 'red']['susc_chi_mass'],
    mode='markers',
    marker=dict(color='red'),
    name='Original Heating Data'
))

fig.add_trace(go.Scatter(
    x=filtered_df[filtered_df['marker_color'] == 'blue']['meas_temp'],
    y=filtered_df[filtered_df['marker_color'] == 'blue']['susc_chi_mass'],
    mode='markers',
    marker=dict(color='blue'),
    name='Original Cooling Data'
))

# Create lines for smoothed data
fig.add_trace(go.Scatter(
    x=filtered_df[filtered_df['marker_color'] == 'red']['meas_temp'],
    y=filtered_df[filtered_df['marker_color'] == 'red']['smoothed_susc_chi_mass'],
    mode='lines',
    line=dict(color='lightcoral', width=2),
    name='Smoothed Heating Data'
))

fig.add_trace(go.Scatter(
    x=filtered_df[filtered_df['marker_color'] == 'blue']['meas_temp'],
    y=filtered_df[filtered_df['marker_color'] == 'blue']['smoothed_susc_chi_mass'],
    mode='lines',
    line=dict(color='lightblue', width=2),
    name='Smoothed Cooling Data'
))

# Customize the layout
fig.update_layout(
    yaxis=dict(tickformat=".2e"),
    legend_title_text='Legend',
    xaxis_title='Temperature (°K)',
    yaxis_title='Magnetic Susceptibility χ [m^3 kg^-1]'
)

# Show the Plotly figure
fig.show()


### ** Not yet viable - LOESS Smoothing

Perform LOESS (Locally Weighted Scatterplot Smoothing) and save the smoothed data

In [ ]:
!pip install statsmodels

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import statsmodels.api as sm

# Define a LOWESS smoothing function for a specific column
def lowess_smoothing(df, column_name, frac):
    lowess = sm.nonparametric.lowess(df[column_name], df.index, frac=frac)
    df['smoothed_' + column_name] = lowess[:, 1]
    return df

# Load your data into a DataFrame, assuming you have a DataFrame named 'filtered_df'

# Get the smoothing fraction (frac) from user input
frac = float(input("Enter the smoothing fraction (0.0 to 1.0) for LOWESS smoothing: "))

# Apply the LOWESS smoothing to 'susc_chi_mass' separately
filtered_df = lowess_smoothing(filtered_df, 'susc_chi_mass', frac)

# Find the maximum value of 'meas_temp'
max_meas_temp = filtered_df['meas_temp'].max()

# Create a new column in the DataFrame to specify marker colors
filtered_df['marker_color'] = 'red'  # Default color is red

# Set color to blue for values with an index greater than the max index
filtered_df.loc[filtered_df.index > filtered_df[filtered_df['meas_temp'] == max_meas_temp].index[0], 'marker_color'] = 'blue'

# Set color back to red for the maximum 'meas_temp' value
filtered_df.loc[filtered_df['meas_temp'] == max_meas_temp, 'marker_color'] = 'red'

# Create a Plotly figure with lines for smoothed data
fig = go.Figure()

# Create scatter plots for original data with color mapping
fig.add_trace(go.Scatter(
    x=filtered_df[filtered_df['marker_color'] == 'red']['meas_temp'],
    y=filtered_df[filtered_df['marker_color'] == 'red']['susc_chi_mass'],
    mode='markers',
    marker=dict(color='red'),
    name='Original Heating Data'
))

fig.add_trace(go.Scatter(
    x=filtered_df[filtered_df['marker_color'] == 'blue']['meas_temp'],
    y=filtered_df[filtered_df['marker_color'] == 'blue']['susc_chi_mass'],
    mode='markers',
    marker=dict(color='blue'),
    name='Original Cooling Data'
))

# Create lines for smoothed data
fig.add_trace(go.Scatter(
    x=filtered_df[filtered_df['marker_color'] == 'red']['meas_temp'],
    y=filtered_df[filtered_df['marker_color'] == 'red']['smoothed_susc_chi_mass'],
    mode='lines',
    line=dict(color='lightcoral', width=2),
    name='Smoothed Heating Data'
))

fig.add_trace(go.Scatter(
    x=filtered_df[filtered_df['marker_color'] == 'blue']['meas_temp'],
    y=filtered_df[filtered_df['marker_color'] == 'blue']['smoothed_susc_chi_mass'],
    mode='lines',
    line=dict(color='lightblue', width=2),
    name='Smoothed Cooling Data'
))

# Customize the layout
fig.update_layout(
    yaxis=dict(tickformat=".2e"),
    legend_title_text='Legend',
    xaxis_title='Temperature (°K)',
    yaxis_title='Magnetic Susceptibility χ [m^3 kg^-1]'
)

# Show the Plotly figure
fig.show()


<a id="plotting"></a>
## Plot the first derivative of the data

Visualizing the first derivative of Magnetic Susceptibility vs. Temperature

In [ ]:
import plotly.express as px
import pandas as pd

# Assuming you have a DataFrame named 'filtered_df'
# Calculate the first derivative
filtered_df['first_derivative'] = filtered_df['susc_chi_mass'].diff()

# Find the maximum value of 'meas_temp'
max_meas_temp = filtered_df['meas_temp'].max()

# Create a new column in the DataFrame to specify marker colors
filtered_df['marker_color'] = 'red'  # Default color is red

# Set color to blue for values with an index greater than the max index
filtered_df.loc[filtered_df.index > filtered_df[filtered_df['meas_temp'] == max_meas_temp].index[0], 'marker_color'] = 'blue'

# Set color back to red for the maximum 'meas_temp' value
filtered_df.loc[filtered_df['meas_temp'] == max_meas_temp, 'marker_color'] = 'red'

# Create a Plotly scatter plot with color mapping for the first derivative
fig = px.scatter(
    filtered_df,
    x='meas_temp',
    y='first_derivative',  # Use the first derivative as the y-axis
    color='marker_color',
    color_discrete_map={'red': 'red', 'blue': 'blue'},
    title=f"First Derivative of Magnetic Susceptibility vs Temperature ({specimen_to_filter})",
    labels={'meas_temp': 'Temperature (°K)', 'first_derivative': 'First Derivative of Susceptibility χ'}
)

# Customize the legend labels
fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
fig.update_traces(showlegend=True, selector=dict(marker_color='red'), name='Heating')
fig.update_traces(showlegend=True, selector=dict(marker_color='blue'), name='Cooling')

# Customize the y-axis to display decimal units
fig.update_layout(yaxis=dict(tickformat=".2e"))  # Set the desired decimal format

# Customize the legend title
fig.update_layout(legend_title_text='Experiment Phase')

# Show the Plotly figure
fig.show()


# Plot the second derivative of the data
Visualizing the second derivative of Magnetic Susceptibility vs. Temperature

In [ ]:
import plotly.express as px
import pandas as pd

# Assuming you have a DataFrame named 'filtered_df'
# Calculate the second derivative
filtered_df['second_derivative'] = filtered_df['susc_chi_mass'].diff().diff()

# Find the maximum value of 'meas_temp'
max_meas_temp = filtered_df['meas_temp'].max()

# Create a new column in the DataFrame to specify marker colors
filtered_df['marker_color'] = 'red'  # Default color is red

# Set color to blue for values with an index greater than the max index
filtered_df.loc[filtered_df.index > filtered_df[filtered_df['meas_temp'] == max_meas_temp].index[0], 'marker_color'] = 'blue'

# Set color back to red for the maximum 'meas_temp' value
filtered_df.loc[filtered_df['meas_temp'] == max_meas_temp, 'marker_color'] = 'red'

# Create a Plotly scatter plot with color mapping for the second derivative
fig = px.scatter(
    filtered_df,
    x='meas_temp',
    y='second_derivative',  # Use the second derivative as the y-axis
    color='marker_color',
    color_discrete_map={'red': 'red', 'blue': 'blue'},
    title=f"Second Derivative of Magnetic Susceptibility vs Temperature ({specimen_to_filter})",
    labels={'meas_temp': 'Temperature (°K)', 'second_derivative': 'Second Derivative of Susceptibility χ'}
)

# Customize the legend labels
fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
fig.update_traces(showlegend=True, selector=dict(marker_color='red'), name='Heating')
fig.update_traces(showlegend=True, selector=dict(marker_color='blue'), name='Cooling')

# Customize the y-axis to display decimal units
fig.update_layout(yaxis=dict(tickformat=".2e"))  # Set the desired decimal format

# Customize the legend title
fig.update_layout(legend_title_text='Experiment Phase')

# Show the Plotly figure
fig.show()


## Two-Tangent Method:

This code allows users to calculate and visualize two tangent lines on a scatter plot representing magnetic susceptibility ('susc_chi_mass') against temperature ('meas_temp').


[Go to References](#References)

In [ ]:
import plotly.graph_objs as go
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import numpy as np
import plotly.offline as pyo

# Assuming you have a DataFrame named 'filtered_df'

# Initialize variables to store the initial data for resetting zoom and tangent
initial_temp_range = [filtered_df['meas_temp'].min(), filtered_df['meas_temp'].max()]
stored_tangent_data = None
second_tangent_data = None

# Create a Plotly scatter plot with color mapping
def create_scatter_plot(temp_range, apply_tangent=False, tangent_data=None, apply_second_tangent=False, second_tangent_data=None, curie_point=None):
    filtered_temp_df = filtered_df[(filtered_df['meas_temp'] >= temp_range[0]) & (filtered_df['meas_temp'] <= temp_range[1])]
    
    fig = go.Figure()  # Create an empty figure
    
    # Add trace for "Heating" points (red)
    heating_df = filtered_temp_df[filtered_temp_df['marker_color'] == 'red']
    fig.add_trace(go.Scatter(
        x=heating_df['meas_temp'],
        y=heating_df['susc_chi_mass'],
        mode='markers',
        name='Heating',  # Set the legend label
        marker=dict(color='red', size=8)
    ))
    
    # Add trace for "Cooling" points (blue)
    cooling_df = filtered_temp_df[filtered_temp_df['marker_color'] == 'blue']
    fig.add_trace(go.Scatter(
        x=cooling_df['meas_temp'],
        y=cooling_df['susc_chi_mass'],
        mode='markers',
        name='Cooling',  # Set the legend label
        marker=dict(color='blue', size=8)
    ))
    
    fig.update_layout(
        title=f"Magnetic Susceptibility vs Temperature ({specimen_to_filter})",
        xaxis=dict(title='Temperature (°K)'),
        yaxis=dict(title='Magnetic Susceptibility χ [m<sup>3</sup> kg<sup>-1</sup>]', tickformat=".2e"),  # Set the desired decimal format
        legend_title_text='Experiment Phase'
    )
    
    # Add the first tangent line to the scatter plot (if apply_tangent is True and tangent_data is provided)
    if apply_tangent and tangent_data:
        fig.add_trace(go.Scatter(x=tangent_data['x'], y=tangent_data['y'], mode='lines', name='Tangent Line', line=dict(color='green', width=2)))
        
        # Extend the tangent line in both directions with dotted lines
        extension_x = np.array([temp_range[0] - 10, temp_range[1] + 10])  # Adjust the range as needed
        extension_y = tangent_data['slope'] * extension_x + tangent_data['intercept']
        
        fig.add_trace(go.Scatter(x=extension_x, y=extension_y, mode='lines', name='Tangent Line Extension', line=dict(color='green', width=2, dash='dot')))
    
    # Add the second tangent line to the scatter plot (if apply_second_tangent is True and second_tangent_data is provided)
    if apply_second_tangent and second_tangent_data:
        fig.add_trace(go.Scatter(x=second_tangent_data['x'], y=second_tangent_data['y'], mode='lines', name='Second Tangent Line', line=dict(color='orange', width=2)))
        
        # Extend the second tangent line in both directions with dotted lines
        extension_x = np.array([temp_range[0] - 10, temp_range[1] + 10])  # Adjust the range as needed
        extension_y = second_tangent_data['slope'] * extension_x + second_tangent_data['intercept']
        
        fig.add_trace(go.Scatter(x=extension_x, y=extension_y, mode='lines', name='Second Tangent Line Extension', line=dict(color='orange', width=2, dash='dot')))
    
    # Customize the y-axis tick format to display decimal units
    fig.update_yaxes(tickformat=".2e")  # Set the desired decimal format
    
    # Customize the axis labels
    fig.update_xaxes(title_text='Temperature (°K)')
    fig.update_yaxes(title_text='Magnetic Susceptibility χ [m<sup>3</sup> kg<sup>-1</sup>]')
    
    # Customize the legend title
    fig.update_layout(legend_title_text='Experiment Phase')
    
    # Add the Curie Point as a scatter point to the figure (if curie_point is provided)
    if curie_point:
        fig.add_trace(go.Scatter(
            x=[curie_point['x']],
            y=[curie_point['y']],
            mode='markers',
            marker=dict(size=10, symbol='circle', color='purple'),
            name='Curie Point'
        ))
    
    return fig

# Define text entry boxes for the start and end points of the temperature range
start_temp_text = widgets.FloatText(
    value=initial_temp_range[0],
    description='Start Temp:',
)
end_temp_text = widgets.FloatText(
    value=initial_temp_range[1],
    description='End Temp:',
)

# Create a button widget to apply the first tangent line
apply_tangent_button = widgets.ToggleButton(value=False, description="Apply Tangent Line")

# Create a button widget to apply the second tangent line
apply_second_tangent_button = widgets.ToggleButton(value=False, description="Apply Second Tangent Line")

# Create a button widget to reset the temperature range
reset_range_button = widgets.Button(description="Reset Range")

# Create an output widget to display the plot
output = widgets.Output()

# Create a button widget to calculate the Curie point
calculate_curie_button = widgets.Button(description="Calculate Curie")

# Define a callback function to update the plot when the text entry values change
def update_plot(change):
    temp_range = [start_temp_text.value, end_temp_text.value]
    apply_tangent = apply_tangent_button.value
    apply_second_tangent = apply_second_tangent_button.value
    fig = create_scatter_plot(temp_range, apply_tangent, stored_tangent_data, apply_second_tangent, second_tangent_data, curie_point=None)
    with output:
        clear_output(wait=True)  # Clear the previous output
        pyo.iplot(fig)  # Use pyo.iplot to display the Plotly figure

# Attach the update_plot function to the text entry boxes' value change event
start_temp_text.observe(update_plot, names='value')
end_temp_text.observe(update_plot, names='value')

# Define a callback function to reset the temperature range
def reset_range(change):
    start_temp_text.value = initial_temp_range[0]
    end_temp_text.value = initial_temp_range[1]
    update_plot(None)

# Attach the reset_range function to the reset_range button's click event
reset_range_button.on_click(reset_range)

# Declare initial_fig as a global variable
initial_fig = None

# Define a callback function to apply and store the first tangent line
def apply_tangent(change):
    global stored_tangent_data, initial_fig
    apply_tangent = apply_tangent_button.value
    if apply_tangent:
        x_range = filtered_df[(filtered_df['meas_temp'] >= start_temp_text.value) & (filtered_df['meas_temp'] <= end_temp_text.value) & (filtered_df['marker_color'] == 'red')]['meas_temp']
        y_range = filtered_df[(filtered_df['meas_temp'] >= start_temp_text.value) & (filtered_df['meas_temp'] <= end_temp_text.value) & (filtered_df['marker_color'] == 'red')]['susc_chi_mass']
        tangent_slope, tangent_intercept = np.polyfit(x_range, y_range, 1)
        tangent_line = tangent_slope * x_range + tangent_intercept
        stored_tangent_data = {'x': x_range, 'y': tangent_line, 'slope': tangent_slope, 'intercept': tangent_intercept}
    
    # Check if initial_fig has been created
    if initial_fig is not None:
        initial_fig = create_scatter_plot([start_temp_text.value, end_temp_text.value], apply_tangent, stored_tangent_data, apply_second_tangent_button.value, second_tangent_data, curie_point=None)
        with output:
            clear_output(wait=True)
            pyo.iplot(initial_fig)
    else:
        update_plot(None)

# Attach the apply_tangent function to the apply_tangent button's change event
apply_tangent_button.observe(apply_tangent, names='value')

# Define a callback function to apply and store the second tangent line
def apply_second_tangent(change):
    global second_tangent_data, initial_fig
    apply_second_tangent = apply_second_tangent_button.value
    if apply_second_tangent:
        x_range = filtered_df[(filtered_df['meas_temp'] >= start_temp_text.value) & (filtered_df['meas_temp'] <= end_temp_text.value) & (filtered_df['marker_color'] == 'red')]['meas_temp']
        y_range = filtered_df[(filtered_df['meas_temp'] >= start_temp_text.value) & (filtered_df['meas_temp'] <= end_temp_text.value) & (filtered_df['marker_color'] == 'red')]['susc_chi_mass']
        tangent_slope, tangent_intercept = np.polyfit(x_range, y_range, 1)
        tangent_line = tangent_slope * x_range + tangent_intercept
        second_tangent_data = {'x': x_range, 'y': tangent_line, 'slope': tangent_slope, 'intercept': tangent_intercept}
    
    # Check if initial_fig has been created
    if initial_fig is not None:
        initial_fig = create_scatter_plot([start_temp_text.value, end_temp_text.value], apply_tangent_button.value, stored_tangent_data, apply_second_tangent, second_tangent_data, curie_point=None)
        with output:
            clear_output(wait=True)
            pyo.iplot(initial_fig)
    else:
        update_plot(None)

# Attach the apply_second_tangent function to the apply_second_tangent button's change event
apply_second_tangent_button.observe(apply_second_tangent, names='value')

# Calculate the intersection point of two lines
def calculate_intersection(tangent1_data, tangent2_data):
    slope1, intercept1 = tangent1_data['slope'], tangent1_data['intercept']
    slope2, intercept2 = tangent2_data['slope'], tangent2_data['intercept']
    
    # Calculate the x-coordinate of the intersection point
    x_intersection = (intercept2 - intercept1) / (slope1 - slope2)
    
    # Calculate the y-coordinate of the intersection point
    y_intersection = slope1 * x_intersection + intercept1
    
    return x_intersection, y_intersection

# Define a callback function to calculate the Curie point
def calculate_curie_point(change):
    global initial_fig
    intersection_point = calculate_intersection(stored_tangent_data, second_tangent_data)
    intersection_x, intersection_y = intersection_point
    intersection_text = f"Curie Point: ({intersection_x:.2f}, {intersection_y:.2e})"
    print(intersection_text)
    
    # Recreate the entire figure with the Curie Point
    initial_fig = create_scatter_plot([start_temp_text.value, end_temp_text.value], apply_tangent_button.value, stored_tangent_data, apply_second_tangent_button.value, second_tangent_data, curie_point={'x': intersection_x, 'y': intersection_y})
    with output:
        clear_output(wait=True)
        pyo.iplot(initial_fig)

# Attach the calculate_curie_point function to the calculate_curie_button's click event
calculate_curie_button.on_click(calculate_curie_point)

# Initialize the plot with initial data
initial_fig = create_scatter_plot(initial_temp_range, apply_tangent_button.value, stored_tangent_data, apply_second_tangent_button.value, second_tangent_data, curie_point=None)

# Set showlegend to False initially
initial_fig.update_layout(showlegend=False)

# Arrange the elements in a VBox including the "Calculate Curie" button
vbox = widgets.VBox([start_temp_text, end_temp_text, apply_tangent_button, apply_second_tangent_button, reset_range_button, calculate_curie_button, output])

# Use pyo.iplot to display the Plotly figure
pyo.iplot(initial_fig)

# Display the VBox containing widgets
display(vbox)


<a id="references"></a>
# References

Fabian https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1029/2012GC004440

Grommé
https://agupubs-onlinelibrary-wiley-com.ezp2.lib.umn.edu/doi/pdf/10.1029/JB074i022p05277